First, get database from server.

1. Make a copy of the old app.db if you want.
2. Double click on clickable script copy_scivocab_db_from_server (in this folder) - that will download the database to your computer.
3. Run the commands below. 

In [ ]:
import pandas as pd
from typing import List, Dict
from sqlalchemy import create_engine

# Increase the maximum number of rows displayed to 999
pd.options.display.max_rows=999

# Connect to the database
engine = create_engine("sqlite:///app.db")

def construct_dfs() -> Dict:
    """Create a dictionary of dataframes, keyed by task name"""
    dfs: Dict = {}

    # Populate the dictionary with the dfs
    dfs["breadth"] = pd.read_sql(
        """select
            child_id,
            word.id as word_id,
            breadth_id,
            strand_id,
            word.target as target_word,
            response_type,
            position,
            timestamp
        from breadth_response
        inner join word on 
        breadth_response.word_id=word.id;""",
        engine,
    )

    dfs["depth"] = pd.read_sql(
        """select
            child_id,
            word.id as word_id,
            depth_id,
            strand_id,
            word.target as target_word,
            image_0,
            image_1,
            image_2,
            image_3,
            timestamp
        from depth_response
        inner join word on 
        depth_response.word_id=word.id;""",
        engine,
    )

    dfs["definition"] = pd.read_sql(
        """select
            child_id,
            word.id as word_id,
            definition_id,
            strand_id,
            word.target as target_word,
            text,
            timestamp
        from definition_response
        inner join word on
        definition_response.word_id=word.id;""",
        engine,
    )

    return dfs


def postprocess_depth_df(df) -> None:
    """Replace d..._X.jpg with X in the image_N columns"""
    for i in range(4):
        df[f"image_{i}"] = (
            df[f"image_{i}"].str.split("_").str[1].str.split(".").str[0]
        )


def calculate_depth_item_score(df):
    """Calculate the score for each depth task response"""
    return (
        sum(
            [
                df[f"image_{index}"] == code
                for index, code in zip((0, 1, 2, 3),
                                       ("a", "b", "c", "e"))
            ]
        )
        / 4
    )

def make_depth_fractions_df(dfs):
    """Make a dataframe showing the fractions of depth task image
    types that the participants got correct"""
    records=[]

    for child_id, df in dfs["depth"].groupby(["child_id"]):
        df = df[df["strand_id"] != "training"]
        record = {"child_id": child_id}
        for index, code in zip((0, 1, 2, 3), ("a", "b", "c", "e")):
            filtered = df[f"image_{index}"] == code
            record.update(
                {f"fraction of {code}'s correct" : filtered.sum() / filtered.count()})
        records.append(record)

    result_df = pd.DataFrame(records)
    return result_df

In [ ]:
dfs = construct_dfs()
postprocess_depth_df(dfs["depth"])
dfs["depth"]["score"] = calculate_depth_item_score(dfs["depth"])

In [ ]:
# Show the breadth task responses
dfs["breadth"]

In [ ]:
# Show the depth task responses
dfs["depth"]

In [ ]:
# Show the definition task responses
dfs["definition"]

In [ ]:
# Make the fractions dataframe
fractions_df=make_depth_fractions_df(dfs)

# Show the fractions dataframe
fractions_df